#Data Cleansing

In [ ]:
#setting
library(haven)

tg <- read_sas('final_project.sas7bdat')

In [ ]:
tg$lung <- ifelse(substr(tg$MAIN_SICK_13,1,1) == 'J' | substr(tg$SUB_SICK_13,1,1) == 'J', 1, 0)
sum(is.na(tg))

[1] 5

In [ ]:
tg$PAST <- as.factor(tg$PAST)
tg$FMLY <- as.factor(tg$FMLY)
tg$SEX <- as.factor(tg$SEX)
tg$SIDO <- as.factor(tg$SIDO)
tg$MOV20_WEK <- as.numeric(tg$MOV20_WEK)
tg$MOV30_WEK <- as.numeric(tg$MOV30_WEK)
tg$WLK30_WEK <- as.numeric(tg$WLK30_WEK)

In [ ]:
#missing 값 제거
tg <- na.omit(tg)

In [ ]:
#"1" -> 0일 , "2" -> 1일이므로 -1씩 변환
tg$MOV20_WEK <-  tg$MOV20_WEK-1
tg$MOV30_WEK <-  tg$MOV30_WEK-1
tg$WLK30_WEK <- tg$WLK30_WEK-1

In [ ]:
#담배를 아예안핀 사람은 control, 담배를 펴본 경험이 있는 사람을 treatment로 두었다.
tg$SMK_STAT_NEW <- ifelse(tg$SMK_STAT == "3" ,1,0)
tg$SMK_STAT_NEW <- as.factor(tg$SMK_STAT_NEW)

In [ ]:
#소득분위를 3단계로 나눔(0~4분위는 1, 5~8분위는 2, 9~10분위는 3)
table(tg$CTRB_PT_TYPE_CD)

CTRB_PT_TYPE_CD <- tg$CTRB_PT_TYPE_CD
table(CTRB_PT_TYPE_CD)

CTRB_PT <- ifelse(CTRB_PT_TYPE_CD %in% c("0","1","2","3","4"), "1", ifelse(CTRB_PT_TYPE_CD %in% c("5","6","7","8"), "2","3"))
table(CTRB_PT)

tg$CTRB_PT <- CTRB_PT


   0    1   10    2    3    4    5    6    7    8    9 
   8  568 1558  466  576  541  631  669  788  841 1083 

CTRB_PT_TYPE_CD
   0    1   10    2    3    4    5    6    7    8    9 
   8  568 1558  466  576  541  631  669  788  841 1083 

CTRB_PT
   1    2    3 
2159 2929 2641 

In [ ]:
#여성 제거(여자의 흡연율이 높지 않기 때문)
tg <- subset(tg, SEX == 1)
tg$SEX <- as.character(tg$SEX)

In [ ]:
#운동관련된 변수 1가지만 채택
tg$MOV30_WEK_NEW <- as.factor(ifelse(tg$MOV30_WEK >= 3, 1, 0))

#Make Propensity score

In [ ]:
prop.model <- glm(SMK_STAT_NEW ~ BMI + PAST + FMLY  + MOV30_WEK_NEW + AGE + SIDO + CTRB_PT , family = binomial, x = TRUE, data = tg)
prop.score <- predict(prop.model, type = "response")
logit.prop <- predict(prop.model)
tg$prop.score <- prop.score
tg$logit.prop <- logit.prop

In [ ]:
summary(prop.model)


Call:
glm(formula = SMK_STAT_NEW ~ BMI + PAST + FMLY + MOV30_WEK_NEW + 
    AGE + SIDO + CTRB_PT, family = binomial, data = tg, x = TRUE)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5194  -0.9913  -0.7852   1.2560   2.1462  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     4.04100    0.38795  10.416  < 2e-16 ***
BMI            -0.08156    0.01048  -7.785 6.98e-15 ***
PAST1          -0.13081    0.16282  -0.803 0.421766    
FMLY1           0.09802    0.09090   1.078 0.280895    
MOV30_WEK_NEW1 -0.38902    0.06982  -5.572 2.52e-08 ***
AGE            -0.04428    0.00448  -9.885  < 2e-16 ***
SIDO26          0.11740    0.12625   0.930 0.352423    
SIDO27          0.44518    0.13985   3.183 0.001457 ** 
SIDO28          0.24664    0.13872   1.778 0.075409 .  
SIDO29          0.11306    0.19510   0.579 0.562262    
SIDO30         -0.01944    0.18060  -0.108 0.914301    
SIDO31          0.64244    0.14765   4.351 1.36e-05 ***
SIDO

#Matching

In [ ]:
install.packages("optmatch")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘SparseM’, ‘abind’, ‘xtable’, ‘svd’, ‘RItools’




In [ ]:
library(optmatch)

Loading required package: survival

The optmatch package has an academic license. Enter relaxinfo() for more information.



#pair matching

In [ ]:
source('functions.R')

## 서울 matching

In [ ]:
tg.seoul <- subset(tg,SIDO %in% c('11'))
seoul.index <- which(tg$SIDO %in% c('11'))

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.seoul$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.seoul$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

# Pair Matching
matchvec=pairmatch(distmat.propensity)
# Note: Can ignore warning message from matching
summary(matchvec)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 

# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.seoul=prop.model$x[seoul.index,-c(1,7:21)];
treatmat.seoul <- Xmat.seoul[treated.subject.index,]
controlmat.seoul.after <- Xmat.seoul[matched.control.subject.index,]

outcome.treatment.seoul <- tg.seoul$lung[treated.subject.index]
outcome.matched.control.seoul <- tg.seoul$lung[matched.control.subject.index]

outcome.seoul <- cbind(outcome.treatment.seoul, outcome.matched.control.seoul)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


Structure of matched sets:
1:1 0:1 
318 379 
Effective Sample Size:  318 
(equivalent number of matched pairs).


##경인 matching

In [ ]:
tg.ki <- subset(tg, SIDO %in% c('41','28'))
ki.index <- which(tg$SIDO %in% c('41','28'))

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.ki$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.ki$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

# Pair Matching
matchvec=pairmatch(distmat.propensity)
# Note: Can ignore warning message from matching
summary(matchvec)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 
# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.ki=prop.model$x[ki.index,-c(1,7:21)];
treatmat.ki <- Xmat.ki[treated.subject.index,]
controlmat.ki.after <- Xmat.ki[matched.control.subject.index,]

outcome.treatment.ki <- tg.ki$lung[treated.subject.index]
outcome.matched.control.ki <- tg.ki$lung[matched.control.subject.index]

outcome.ki <- cbind(outcome.treatment.ki, outcome.matched.control.ki)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


Structure of matched sets:
1:1 0:1 
597 305 
Effective Sample Size:  597 
(equivalent number of matched pairs).


##충청 matching


In [ ]:
tg.cc <- subset(tg, SIDO %in% c('43','44','30'))
cc.index <- which(tg$SIDO %in% c('43','44','30'))

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.cc$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.cc$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

# Pair Matching
matchvec=pairmatch(distmat.propensity)
# Note: Can ignore warning message from matching
summary(matchvec)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 
# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.cc=prop.model$x[cc.index,-c(1,7:21)];
treatmat.cc <- Xmat.cc[treated.subject.index,]
controlmat.cc.after <- Xmat.cc[matched.control.subject.index,]

outcome.treatment.cc <- tg.cc$lung[treated.subject.index]
outcome.matched.control.cc <- tg.cc$lung[matched.control.subject.index]

outcome.cc <- cbind(outcome.treatment.cc, outcome.matched.control.cc)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


Structure of matched sets:
1:1 0:1 
225 138 
Effective Sample Size:  225 
(equivalent number of matched pairs).


##호남,제주 matching

In [ ]:
tg.hn <- subset(tg, SIDO %in% c('45','46','49','29'))
hn.index <- which(tg$SIDO %in% c('45','46','29','49'))

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.hn$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.hn$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

# Pair Matching
matchvec=pairmatch(distmat.propensity)
# Note: Can ignore warning message from matching
summary(matchvec)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 
# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.hn=prop.model$x[hn.index,-c(1,7:21)];
treatmat.hn <- Xmat.hn[treated.subject.index,]
controlmat.hn.after <- Xmat.hn[matched.control.subject.index,]

outcome.treatment.hn <- tg.hn$lung[treated.subject.index]
outcome.matched.control.hn <- tg.hn$lung[matched.control.subject.index]

outcome.hn <- cbind(outcome.treatment.hn, outcome.matched.control.hn)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


Structure of matched sets:
1:1 0:1 
185 135 
Effective Sample Size:  185 
(equivalent number of matched pairs).


##경북,강원 matching

In [ ]:
tg.kg <- subset(tg, SIDO %in% c('42','47','27'))
kg.index <- which(tg$SIDO %in% c('42','47','27'))

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.kg$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.kg$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

# Pair Matching
matchvec=pairmatch(distmat.propensity)
# Note: Can ignore warning message from matching
summary(matchvec)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 
# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.kg=prop.model$x[kg.index,-c(1,7:21)];
treatmat.kg <- Xmat.kg[treated.subject.index,]
controlmat.kg.after <- Xmat.kg[matched.control.subject.index,]

outcome.treatment.kg <- tg.kg$lung[treated.subject.index]
outcome.matched.control.kg <- tg.kg$lung[matched.control.subject.index]

outcome.kg <- cbind(outcome.treatment.kg, outcome.matched.control.kg)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


Structure of matched sets:
1:1 0:1 
319 128 
Effective Sample Size:  319 
(equivalent number of matched pairs).


## 경남 matching

In [ ]:
tg.kn <- subset(tg, SIDO %in% c('26','31','48'))
kn.index <- which(tg$SIDO %in% c('26','31','48'))

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.kn$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.kn$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

# Pair Matching
matchvec=pairmatch(distmat.propensity)
# Note: Can ignore warning message from matching
summary(matchvec)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 
# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.kn=prop.model$x[kn.index,-c(1,7:21)];
treatmat.kn <- Xmat.kn[treated.subject.index,]
controlmat.kn.after <- Xmat.kn[matched.control.subject.index,]

outcome.treatment.kn <- tg.kn$lung[treated.subject.index]
outcome.matched.control.kn <- tg.kn$lung[matched.control.subject.index]

outcome.kn <- cbind(outcome.treatment.kn, outcome.matched.control.kn)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


Structure of matched sets:
1:1 0:1 
413 223 
Effective Sample Size:  413 
(equivalent number of matched pairs).


##covariates differences

In [ ]:
treatmat <- rbind(treatmat.seoul, treatmat.ki,treatmat.cc,treatmat.hn,treatmat.kg,treatmat.kn)
controlmat.before <- prop.model$x[tg$SMK_STAT_NEW == 0,-c(1,7:21)]
controlmat.after <- rbind(controlmat.seoul.after, controlmat.ki.after, controlmat.cc.after, controlmat.hn.after, controlmat.kg.after, controlmat.kn.after)

controlmean.before=apply(controlmat.before,2,mean);
treatmean=apply(treatmat,2,mean);
treatvar=apply(treatmat,2,var);
controlvar=apply(controlmat.before,2,var);
stand.diff.before=(treatmean-controlmean.before)/sqrt((treatvar+controlvar)/2);
# Standardized differences after matching
controlmean.after=apply(controlmat.after,2,mean);
# Standardized differences after matching
stand.diff.after=(treatmean-controlmean.after)/sqrt((treatvar+controlvar)/2);

round(cbind(stand.diff.before, stand.diff.after),3)


,stand.diff.before,stand.diff.after
BMI,-0.212,-0.067
PAST1,-0.041,-0.011
FMLY1,0.031,0.003
MOV30_WEK_NEW1,-0.190,-0.022
AGE,-0.265,0.051
CTRB_PT2,0.067,0.017
CTRB_PT3,-0.134,-0.072


#Test for ATE

In [ ]:
outcome.pair <- data.frame(rbind(outcome.seoul, outcome.ki, outcome.cc, outcome.hn, outcome.kg, outcome.kn))
colnames(outcome.pair) <- c("treatment","matched.control")

'data.frame':	2057 obs. of  2 variables:
 $ treatment      : num  0 0 0 0 0 0 0 0 0 1 ...
 $ matched.control: num  0 0 1 0 0 0 0 0 0 0 ...


In [ ]:
treatment <- c(1,1,0,0)
control <- c(1,0,1,0)


cross.pair <- matrix(rep(0,4), nrow = 2, ncol = 2)
for(i in 1:nrow(outcome.pair)) {
  if (outcome.pair$treatment[i] == 1 & outcome.pair$matched.control[i] == 1) {
    cross.pair[1,1] = cross.pair[1,1] + 1
  } else if (outcome.pair$treatment[i] == 1 & outcome.pair$matched.control[i] == 0) {
    cross.pair[1,2] = cross.pair[1,2] + 1
  } else if (outcome.pair$treatment[i] == 0 & outcome.pair$matched.control[i] == 1) {
    cross.pair[2,1] = cross.pair[2,1] + 1
  } else {
    cross.pair[2,2] = cross.pair[2,2] + 1
  }
}

count <- c(cross.pair[1,1],cross.pair[1,2],cross.pair[2,1],cross.pair[2,2])
pair <- data.frame(treatment,control,count)
tab <- xtabs(count~treatment+control,data = pair)

mcnemar.test(tab,correct = T)


	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 0.4962, df = 1, p-value = 0.4812


#matching with replacement

##서울 matching

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.seoul$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.seoul$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

treatment.index <- as.numeric(rownames(distmat.propensity))
matched.control.index <- c(rep(0,sum(treated)))
for (i in c(1:sum(treated))) {
  col <- which.min(distmat.propensity[i,]) 
  matched.control.index[i] <- as.numeric(colnames(distmat.propensity)[col])
}

cat('treatment:' , length(treatment.index),'\n')
cat('matched.control.index : ' , length(matched.control.index))

Xmat.seoul=prop.model$x[seoul.index,-c(1,7:21)];
treatmat.seoul <- Xmat.seoul[treatment.index,]
controlmat.seoul.after <- Xmat.seoul[matched.control.index,]

outcome.treatment.seoul <- tg.seoul$lung[treatment.index]
outcome.matched.control.seoul <- tg.seoul$lung[matched.control.index]

outcome.seoul <- cbind(outcome.treatment.seoul, outcome.matched.control.seoul)

treatment: 318 
matched.control.index :  318

##경인 matching

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.ki$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.ki$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

treatment.index <- as.numeric(rownames(distmat.propensity))
matched.control.index <- c(rep(0,sum(treated)))
for (i in c(1:sum(treated))) {
  col <- which.min(distmat.propensity[i,]) 
  matched.control.index[i] <- as.numeric(colnames(distmat.propensity)[col])
}

cat('treatment:' , length(treatment.index),'\n')
cat('matched.control.index : ' , length(matched.control.index))

Xmat.ki=prop.model$x[ki.index,-c(1,7:21)];
treatmat.ki <- Xmat.ki[treatment.index,]
controlmat.ki.after <- Xmat.ki[matched.control.index,]

outcome.treatment.ki <- tg.ki$lung[treatment.index]
outcome.matched.control.ki <- tg.ki$lung[matched.control.index]

outcome.ki <- cbind(outcome.treatment.ki, outcome.matched.control.ki)

treatment: 597 
matched.control.index :  597

##충청 matching

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.cc$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.cc$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

treatment.index <- as.numeric(rownames(distmat.propensity))
matched.control.index <- c(rep(0,sum(treated)))
for (i in c(1:sum(treated))) {
  col <- which.min(distmat.propensity[i,]) 
  matched.control.index[i] <- as.numeric(colnames(distmat.propensity)[col])
}

cat('treatment:' , length(treatment.index),'\n')
cat('matched.control.index : ' , length(matched.control.index))

Xmat.cc=prop.model$x[cc.index,-c(1,7:21)];
treatmat.cc <- Xmat.cc[treatment.index,]
controlmat.cc.after <- Xmat.cc[matched.control.index,]

outcome.treatment.cc <- tg.cc$lung[treatment.index]
outcome.matched.control.cc <- tg.cc$lung[matched.control.index]

outcome.cc <- cbind(outcome.treatment.cc, outcome.matched.control.cc)

treatment: 225 
matched.control.index :  225

##호남,제주 matching

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.hn$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.hn$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

treatment.index <- as.numeric(rownames(distmat.propensity))
matched.control.index <- c(rep(0,sum(treated)))
for (i in c(1:sum(treated))) {
  col <- which.min(distmat.propensity[i,]) 
  matched.control.index[i] <- as.numeric(colnames(distmat.propensity)[col])
}

cat('treatment:' , length(treatment.index),'\n')
cat('matched.control.index : ' , length(matched.control.index))

Xmat.hn=prop.model$x[hn.index,-c(1,7:21)];
treatmat.hn <- Xmat.hn[treatment.index,]
controlmat.hn.after <- Xmat.hn[matched.control.index,]

outcome.treatment.hn <- tg.hn$lung[treatment.index]
outcome.matched.control.hn <- tg.hn$lung[matched.control.index]

outcome.hn <- cbind(outcome.treatment.hn, outcome.matched.control.hn)

treatment: 185 
matched.control.index :  185

##경북,강원 matching

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.kg$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.kg$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

treatment.index <- as.numeric(rownames(distmat.propensity))
matched.control.index <- c(rep(0,sum(treated)))
for (i in c(1:sum(treated))) {
  col <- which.min(distmat.propensity[i,]) 
  matched.control.index[i] <- as.numeric(colnames(distmat.propensity)[col])
}

cat('treatment:' , length(treatment.index),'\n')
cat('matched.control.index : ' , length(matched.control.index))

Xmat.kg=prop.model$x[kg.index,-c(1,7:21)];
treatmat.kg <- Xmat.kg[treatment.index,]
controlmat.kg.after <- Xmat.kg[matched.control.index,]

outcome.treatment.kg <- tg.kg$lung[treatment.index]
outcome.matched.control.kg <- tg.kg$lung[matched.control.index]

outcome.kg <- cbind(outcome.treatment.kg, outcome.matched.control.kg)

treatment: 319 
matched.control.index :  319

##경남 matching

In [ ]:
# Find logit(propensity score);
logit.propscore <- tg.kn$logit.prop
# Construct a distance matrix which gives the absolute difference between the 
# propensity scores of the treated and control subjects
treated <- tg.kn$SMK_STAT_NEW == 1
distmat.propensity=matrix(rep(0,sum(treated==1)*sum(treated==0)),nrow=sum(treated==1));
for(i in 1:sum(treated==1)){
  for(j in 1:sum(treated==0)){
    distmat.propensity[i,j]=abs((logit.propscore[treated==1])[i]-(logit.propscore[treated==0])[j]);
  }
}
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat.propensity)=subject.index[treated==1]
colnames(distmat.propensity)=subject.index[treated==0]

treatment.index <- as.numeric(rownames(distmat.propensity))
matched.control.index <- c(rep(0,sum(treated)))
for (i in c(1:sum(treated))) {
  col <- which.min(distmat.propensity[i,]) 
  matched.control.index[i] <- as.numeric(colnames(distmat.propensity)[col])
}

cat('treatment:' , length(treatment.index),'\n')
cat('matched.control.index : ' , length(matched.control.index))

Xmat.kn=prop.model$x[kn.index,-c(1,7:21)];
treatmat.kn <- Xmat.kn[treatment.index,]
controlmat.kn.after <- Xmat.kn[matched.control.index,]

outcome.treatment.kn <- tg.kn$lung[treatment.index]
outcome.matched.control.kn <- tg.kn$lung[matched.control.index]

outcome.kn <- cbind(outcome.treatment.kn, outcome.matched.control.kn)

treatment: 413 
matched.control.index :  413

##covariates differences

In [ ]:
treatmat <- rbind(treatmat.seoul, treatmat.ki,treatmat.cc,treatmat.hn,treatmat.kg,treatmat.kn)
controlmat.before <- prop.model$x[tg$SMK_STAT_NEW == 0,-c(1,7:21)]
controlmat.after <- rbind(controlmat.seoul.after, controlmat.ki.after, controlmat.cc.after, controlmat.hn.after, controlmat.kg.after, controlmat.kn.after)

controlmean.before=apply(controlmat.before,2,mean);
treatmean=apply(treatmat,2,mean);
treatvar=apply(treatmat,2,var);
controlvar=apply(controlmat.before,2,var);
stand.diff.before=(treatmean-controlmean.before)/sqrt((treatvar+controlvar)/2);
# Standardized differences after matching
controlmean.after=apply(controlmat.after,2,mean);
# Standardized differences after matching
stand.diff.after=(treatmean-controlmean.after)/sqrt((treatvar+controlvar)/2);

round(cbind(stand.diff.before, stand.diff.after),3)

,stand.diff.before,stand.diff.after
BMI,-0.212,0.000
PAST1,-0.041,-0.021
FMLY1,0.031,0.012
MOV30_WEK_NEW1,-0.190,-0.020
AGE,-0.265,0.014
CTRB_PT2,0.067,-0.004
CTRB_PT3,-0.134,-0.004


#Test for ATE

In [ ]:
outcome.pair <- data.frame(rbind(outcome.seoul, outcome.ki, outcome.cc, outcome.hn, outcome.kg, outcome.kn))
colnames(outcome.pair) <- c("treatment","matched.control")

treatment <- c(1,1,0,0)
control <- c(1,0,1,0)


cross.pair <- matrix(rep(0,4), nrow = 2, ncol = 2)
for(i in 1:nrow(outcome.pair)) {
  if (outcome.pair$treatment[i] == 1 & outcome.pair$matched.control[i] == 1) {
    cross.pair[1,1] = cross.pair[1,1] + 1
  } else if (outcome.pair$treatment[i] == 1 & outcome.pair$matched.control[i] == 0) {
    cross.pair[1,2] = cross.pair[1,2] + 1
  } else if (outcome.pair$treatment[i] == 0 & outcome.pair$matched.control[i] == 1) {
    cross.pair[2,1] = cross.pair[2,1] + 1
  } else {
    cross.pair[2,2] = cross.pair[2,2] + 1
  }
}

count <- c(cross.pair[1,1],cross.pair[1,2],cross.pair[2,1],cross.pair[2,2])
pair <- data.frame(treatment,control,count)
tab <- xtabs(count~treatment+control,data = pair)

mcnemar.test(tab,correct = T)



	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 0.29803, df = 1, p-value = 0.5851


#mahalanobis-caliper 1000 matching


##서울 matching


In [ ]:
# Matrix of covariates, excluding intercept
Xmat=prop.model$x[seoul.index,-c(1,7:21)];
# Rank based Mahalanobis distance
treated <- tg.seoul$SMK_STAT_NEW == 1
distmat=smahal(treated,Xmat)
# Add caliper
logit.propscore=tg.seoul$logit.prop
distmat2=addcaliper(distmat,treated,logit.propscore)
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat2)=subject.index[treated==1]
colnames(distmat2)=subject.index[treated==0]

matchvec=pairmatch(distmat2)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 

# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.seoul=prop.model$x[seoul.index,-c(1,7:21)];
treatmat.seoul <- Xmat.seoul[treated.subject.index,]
controlmat.seoul.after <- Xmat.seoul[matched.control.subject.index,]

outcome.treatment.seoul <- tg.seoul$lung[treated.subject.index]
outcome.matched.control.seoul <- tg.seoul$lung[matched.control.subject.index]

outcome.seoul <- cbind(outcome.treatment.seoul, outcome.matched.control.seoul)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


##경인 matching

In [ ]:
# Matrix of covariates, excluding intercept
Xmat=prop.model$x[ki.index,-c(1,7:21)];
# Rank based Mahalanobis distance
treated <- tg.ki$SMK_STAT_NEW == 1
distmat=smahal(treated,Xmat)
# Add caliper
logit.propscore=tg.ki$logit.prop
distmat2=addcaliper(distmat,treated,logit.propscore)
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat2)=subject.index[treated==1]
colnames(distmat2)=subject.index[treated==0]

matchvec=pairmatch(distmat2)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 

# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.ki=prop.model$x[ki.index,-c(1,7:21)];
treatmat.ki <- Xmat.ki[treated.subject.index,]
controlmat.ki.after <- Xmat.ki[matched.control.subject.index,]

outcome.treatment.ki <- tg.ki$lung[treated.subject.index]
outcome.matched.control.ki <- tg.ki$lung[matched.control.subject.index]

outcome.ki <- cbind(outcome.treatment.ki, outcome.matched.control.ki)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


##충청 matching

In [ ]:
# Matrix of covariates, excluding intercept
Xmat=prop.model$x[cc.index,-c(1,7:21)];
# Rank based Mahalanobis distance
treated <- tg.cc$SMK_STAT_NEW == 1
distmat=smahal(treated,Xmat)
# Add caliper
logit.propscore=tg.cc$logit.prop
distmat2=addcaliper(distmat,treated,logit.propscore)
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat2)=subject.index[treated==1]
colnames(distmat2)=subject.index[treated==0]

matchvec=pairmatch(distmat2)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 

# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.cc=prop.model$x[cc.index,-c(1,7:21)];
treatmat.cc <- Xmat.cc[treated.subject.index,]
controlmat.cc.after <- Xmat.cc[matched.control.subject.index,]

outcome.treatment.cc <- tg.cc$lung[treated.subject.index]
outcome.matched.control.cc <- tg.cc$lung[matched.control.subject.index]

outcome.cc <- cbind(outcome.treatment.cc, outcome.matched.control.cc)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


##호남,제주 matching

In [ ]:
# Matrix of covariates, excluding intercept
Xmat=prop.model$x[hn.index,-c(1,7:21)];
# Rank based Mahalanobis distance
treated <- tg.hn$SMK_STAT_NEW == 1
distmat=smahal(treated,Xmat)
# Add caliper
logit.propscore=tg.hn$logit.prop
distmat2=addcaliper(distmat,treated,logit.propscore)
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat2)=subject.index[treated==1]
colnames(distmat2)=subject.index[treated==0]

matchvec=pairmatch(distmat2)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 

# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.hn=prop.model$x[hn.index,-c(1,7:21)];
treatmat.hn <- Xmat.hn[treated.subject.index,]
controlmat.hn.after <- Xmat.hn[matched.control.subject.index,]

outcome.treatment.hn <- tg.hn$lung[treated.subject.index]
outcome.matched.control.hn <- tg.hn$lung[matched.control.subject.index]

outcome.hn <- cbind(outcome.treatment.hn, outcome.matched.control.hn)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


##경북, 강원 matching

In [ ]:
# Matrix of covariates, excluding intercept
Xmat=prop.model$x[kg.index,-c(1,7:21)];
# Rank based Mahalanobis distance
treated <- tg.kg$SMK_STAT_NEW == 1
distmat=smahal(treated,Xmat)
# Add caliper
logit.propscore=tg.kg$logit.prop
distmat2=addcaliper(distmat,treated,logit.propscore)
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat2)=subject.index[treated==1]
colnames(distmat2)=subject.index[treated==0]

matchvec=pairmatch(distmat2)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 

# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.kg=prop.model$x[kg.index,-c(1,7:21)];
treatmat.kg <- Xmat.kg[treated.subject.index,]
controlmat.kg.after <- Xmat.kg[matched.control.subject.index,]

outcome.treatment.kg <- tg.kg$lung[treated.subject.index]
outcome.matched.control.kg <- tg.kg$lung[matched.control.subject.index]

outcome.kg <- cbind(outcome.treatment.kg, outcome.matched.control.kg)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


##경남 matching

In [ ]:
# Matrix of covariates, excluding intercept
Xmat=prop.model$x[kn.index,-c(1,7:21)];
# Rank based Mahalanobis distance
treated <- tg.kn$SMK_STAT_NEW == 1
distmat=smahal(treated,Xmat)
# Add caliper
logit.propscore=tg.kn$logit.prop
distmat2=addcaliper(distmat,treated,logit.propscore)
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,length(treated),1)
rownames(distmat2)=subject.index[treated==1]
colnames(distmat2)=subject.index[treated==0]

matchvec=pairmatch(distmat2)

# Create vectors of the subject indices of the treatment units ordered by
# their matched set and corresponding control unit
treated.subject.index=rep(0,sum(treated==1))
matched.control.subject.index=rep(0,length(treated.subject.index))
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) # The subject indices in 

# the order of matchvec
for(i in 1:length(treated.subject.index)){
  matched.set.temp=which(matchedset.index.numeric==i)
  matched.set.temp.indices=subjects.match.order[matched.set.temp]
  if(treated[matched.set.temp.indices[1]]==1){
    treated.subject.index[i]=matched.set.temp.indices[1]
    matched.control.subject.index[i]=matched.set.temp.indices[2]
  }
  if(treated[matched.set.temp.indices[2]]==1){
    treated.subject.index[i]=matched.set.temp.indices[2]
    matched.control.subject.index[i]=matched.set.temp.indices[1]
  }
}

Xmat.kn=prop.model$x[kn.index,-c(1,7:21)];
treatmat.kn <- Xmat.kn[treated.subject.index,]
controlmat.kn.after <- Xmat.kn[matched.control.subject.index,]

outcome.treatment.kn <- tg.kn$lung[treated.subject.index]
outcome.matched.control.kn <- tg.kn$lung[matched.control.subject.index]

outcome.kn <- cbind(outcome.treatment.kn, outcome.matched.control.kn)

Warning message in fullmatch(x = x, min.controls = controls, max.controls = controls, :
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


##covariates differences

In [ ]:
treatmat <- rbind(treatmat.seoul, treatmat.ki,treatmat.cc,treatmat.hn,treatmat.kg,treatmat.kn)
controlmat.before <- prop.model$x[tg$SMK_STAT_NEW == 0,-c(1,7:21)]
controlmat.after <- rbind(controlmat.seoul.after, controlmat.ki.after, controlmat.cc.after, controlmat.hn.after, controlmat.kg.after, controlmat.kn.after)

controlmean.before=apply(controlmat.before,2,mean);
treatmean=apply(treatmat,2,mean);
treatvar=apply(treatmat,2,var);
controlvar=apply(controlmat.before,2,var);
stand.diff.before=(treatmean-controlmean.before)/sqrt((treatvar+controlvar)/2);
# Standardized differences after matching
controlmean.after=apply(controlmat.after,2,mean);
# Standardized differences after matching
stand.diff.after=(treatmean-controlmean.after)/sqrt((treatvar+controlvar)/2);

round(cbind(stand.diff.before, stand.diff.after),3)

,stand.diff.before,stand.diff.after
BMI,-0.212,-0.061
PAST1,-0.041,0.067
FMLY1,0.031,0.081
MOV30_WEK_NEW1,-0.190,0.010
AGE,-0.265,-0.002
CTRB_PT2,0.067,0.005
CTRB_PT3,-0.134,-0.044


#Test for ATE

In [ ]:
outcome.ma1000 <- data.frame(rbind(outcome.seoul, outcome.ki, outcome.cc, outcome.hn, outcome.kg, outcome.kn))
colnames(outcome.ma1000) <- c("treatment","matched.control")

treatment <- c(1,1,0,0)
control <- c(1,0,1,0)


cross.ma1000 <- matrix(rep(0,4), nrow = 2, ncol = 2)
for(i in 1:nrow(outcome.ma1000)) {
  if (outcome.ma1000$treatment[i] == 1 & outcome.ma1000$matched.control[i] == 1) {
    cross.ma1000[1,1] = cross.ma1000[1,1] + 1
  } else if (outcome.ma1000$treatment[i] == 1 & outcome.ma1000$matched.control[i] == 0) {
    cross.ma1000[1,2] = cross.ma1000[1,2] + 1
  } else if (outcome.ma1000$treatment[i] == 0 & outcome.ma1000$matched.control[i] == 1) {
    cross.ma1000[2,1] = cross.ma1000[2,1] + 1
  } else {
    cross.ma1000[2,2] = cross.ma1000[2,2] + 1
  }
}

count <- c(cross.ma1000[1,1],cross.ma1000[1,2],cross.ma1000[2,1],cross.ma1000[2,2])
ma1000 <- data.frame(treatment,control,count)
tab <- xtabs(count~treatment+control,data = ma1000)

mcnemar.test(tab,correct = T)


	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 2.5929, df = 1, p-value = 0.1073


#Full Matching



In [ ]:
# Matrix of covariates, excluding intercept
Xmat=prop.model$x[,-c(1,7:21)]
treatment=(tg$SMK_STAT_NEW == 1)
# Rank based Mahalanobis distance
distmat=smahal(treatment,Xmat)
# Add caliper
logit.propscore=tg$logit.prop
distmat2=addcaliper(distmat,treatment,logit.propscore)
### Create a subject index and name the rows and columns of distance matrix by ### this subject index
subject.index=seq(1,nrow(Xmat),1)
rownames(distmat2)=subject.index[treatment==1]
colnames(distmat2)=subject.index[treatment==0]


library(optmatch)
matchvec=fullmatch(distmat2)

# The subject indices in the order of matchvec
matchedset.index=substr(matchvec,start=3,stop=10)
matchedset.index.numeric=as.numeric(matchedset.index)
subjects.match.order=as.numeric(names(matchvec)) 

# Create a numeric variable for which stratum each unit belongs to
# 0 denotes that the unit was not matched
stratum.short=substr(matchvec,start=3,stop=10);
stratum.numeric=as.numeric(stratum.short);
# Reassign numbers to each stratum that go from 1,..., no. of straum
sort.unique.stratum=sort(unique(stratum.numeric));
stratum.myindex.matchvecorder=rep(0,length(stratum.numeric));
for(i in 1:length(sort.unique.stratum)){
  stratum.myindex.matchvecorder[stratum.numeric==sort.unique.stratum[i]]=i;
}

stratum.myindex=rep(0,length(stratum.myindex.matchvecorder))
stratum.myindex[subjects.match.order]=stratum.myindex.matchvecorder

stratumStructure(matchvec)
summary(matchvec)



Warning message in fullmatch(distmat2):
“Without 'data' argument the order of the match is not guaranteed
    to be the same as your original data.”


 7:1  4:1  3:1  2:1  1:1  1:2  1:3  1:4  1:5  1:6  1:7  1:8  1:9 1:10 1:11 1:13 
   3   15   43  139  884  287  177  101   56   28   13   12    3    3    1    1 
1:17 1:22 
   2    1 

Structure of matched sets:
5+:1  4:1  3:1  2:1  1:1  1:2  1:3  1:4 1:5+ 
   3   15   43  139  884  287  177  101  120 
Effective Sample Size:  2178.5 
(equivalent number of matched pairs).


In [ ]:
# Calculate the standardized differences
Xmat.matchorder=Xmat[subjects.match.order,]

std.diff.before=rep(0,ncol(Xmat.matchorder));
std.diff.after=rep(0,ncol(Xmat.matchorder));
names(std.diff.before)=names(Xmat.matchorder[1,]);
names(std.diff.after)=names(Xmat.matchorder[1,]);
for(i in 1:ncol(Xmat.matchorder)){
  temp.stand.diff=standardized.diff.func(Xmat[,i],treatment,stratum.myindex);
  std.diff.before[i]=temp.stand.diff$std.diff.before.matching;
  std.diff.after[i]=temp.stand.diff$std.diff.after.matching;
}

fullmatch.std.diff = round(cbind(std.diff.before,std.diff.after), 3)

In [ ]:
fullmatch.std.diff

,std.diff.before,std.diff.after
BMI,-0.212,-0.009
PAST1,-0.041,-0.052
FMLY1,0.031,-0.022
MOV30_WEK_NEW1,-0.190,0.001
AGE,-0.265,-0.001
CTRB_PT2,0.067,0.001
CTRB_PT3,-0.134,0.001


# #Sensitivity Analysis - mahalanobis-caliper 1000 matching

In [ ]:
sens.analysis.mcnemar=function(D,Tobs,Gamma){
  p.positive = Gamma/(1+Gamma);
  p.negative = 1/(1+Gamma);
  lower.bound = 1-pbinom(Tobs-1,D,p.negative);
  upper.bound = 1-pbinom(Tobs-1,D,p.positive);
  
  exp.p.positive = p.positive*D
  exp.p.negative = p.negative*D
  var.p.positive = p.positive*(1-p.positive)*D
  var.p.negative = p.negative*(1-p.negative)*D
  upper.bound.approx = 1 - pnorm((Tobs-exp.p.positive)/sqrt(var.p.positive))
  lower.bound.approx = 1 - pnorm((Tobs-exp.p.negative)/sqrt(var.p.negative))
  
  list(lower=lower.bound,upper=upper.bound,
       lower.appx = lower.bound.approx,
       upper.appx = upper.bound.approx);
}

## 
Gamma.grid = seq(1, 2, 0.1)
lower.b = upper.b = rep(NA, length(Gamma.grid))
for(i in 1:length(Gamma.grid)){
  sens.res = sens.analysis.mcnemar(D = 227+193, Tobs = 193, Gamma = Gamma.grid[i])
  lower.b[i] = sens.res$lower
  upper.b[i] = sens.res$upper
}
round(cbind(lower.b, upper.b), 4)

lower.b,upper.b
0.9562,0.9562
0.7680,0.9964
0.4375,0.9998
0.1651,1.0000
0.0421,1.0000
0.0076,1.0000
0.0010,1.0000
0.0001,1.0000
0.0000,1.0000
0.0000,1.0000
